In [11]:
import requests
import json
import sqlite3

In [40]:
conn = sqlite3.connect("/home/xiaoyu/IdeaProjects/argus/main.db")
cur = conn.cursor()
res = cur.execute("select distinct(bondCode) from issuance")
p = {"bondType":999999,"pageNo":1,"pageSize":25}
u = "https://www.chinamoney.com.cn/ags/ms/cm-u-bond-md/BondMarketInfoList2"

from datetime import datetime
import time

def splitRecords(current):
    res = cur.execute("select distinct(bondCode) from issuance")
    existing_codes = set(res.fetchall())
    new = [ x for x in current if (x['bondCode'] not in existing_codes) ]
    if len(new)==len(current):
        return (new,False) # need to fetch next page
    else:
        return (new,True)

#'bondDefinedCode': 'dhgghhea6y',
#'bondName': '22瑞泽2优先',
#'bondCode': '2289223',
#'issueStartDate': '2022-08-29',
#'issueEndDate': '2022-08-29',
#'bondTypeCode': '999999',
#'bondType': '资产支持证券',
#'entyFullName': '安徽国元信托有限责任公司',
#'entyDefinedCode': '102793',
#'debtRtng': 'AAA',
#'isin': None,
#'inptTp': '0'    
    
    
def saveToDb(ds):
    for d in ds:
        print("inserting",d['bondName'])
        fs = "'"+"','".join(d.keys())+"'"
        vs = "'"+"','".join(d.values())+"'"
        cur.execute(f"INSERT INTO issuance ({fs},'ts') VALUES (,{str(datetime.now())})")
    conn.commit()
        

def fetch_page(pg,acc=None):
    p.update({"pageNo":pg})
    try:
        r = requests.post(u,p)
        time.sleep(3)
        result = json.loads(r.text)['data']['resultList']
        
    except Exception as e:
        print(r.text)
        print("Error",e)
    except JSONDecodeError as e:
        print("Error",r.text,r)
        
    left,enough = splitRecords(result)
    
    if enough:
        saveToDb(left+acc)
        return True
    else:
        fetch_page(pg+1,left)

In [41]:
fetch_page(1)

There are too many connections from your internet address[2409:8a1e:1a73:e520:3539:720:c5e8:e500] to url [/ags/ms/cm-u-bond-md/BondMarketInfoList2]

Error Expecting value: line 1 column 1 (char 0)


UnboundLocalError: local variable 'result' referenced before assignment

In [10]:
json.loads(r.text)['data']['resultList']

[{'bondDefinedCode': 'dhgghhea6y',
  'bondName': '22瑞泽2优先',
  'bondCode': '2289223',
  'issueStartDate': '2022-08-29',
  'issueEndDate': '2022-08-29',
  'bondTypeCode': '999999',
  'bondType': '资产支持证券',
  'entyFullName': '安徽国元信托有限责任公司',
  'entyDefinedCode': '102793',
  'debtRtng': 'AAA',
  'isin': None,
  'inptTp': '0'},
 {'bondDefinedCode': 'dhgggiwyyz',
  'bondName': '22瑞泽2C',
  'bondCode': '2289224',
  'issueStartDate': '2022-08-29',
  'issueEndDate': '2022-08-29',
  'bondTypeCode': '999999',
  'bondType': '资产支持证券',
  'entyFullName': '安徽国元信托有限责任公司',
  'entyDefinedCode': '102793',
  'debtRtng': '---',
  'isin': None,
  'inptTp': '0'},
 {'bondDefinedCode': 'dhgfhptwj4',
  'bondName': '22常兴业2优先A1',
  'bondCode': '2289212',
  'issueStartDate': '2022-08-25',
  'issueEndDate': '2022-08-25',
  'bondTypeCode': '999999',
  'bondType': '资产支持证券',
  'entyFullName': '中建投信托股份有限公司(原：中建投信托有限责任公司)',
  'entyDefinedCode': '103006',
  'debtRtng': 'AAA',
  'isin': None,
  'inptTp': '0'},
 {'bondDefinedC

In [ ]:
https://www.chinamoney.com.cn/chinese/zqjc/?bondDefinedCode=dhgghhea6y

In [56]:
u = "https://www.chinamoney.com.cn/ags/ms/cm-u-bond-md/BondDetailInfo"

conn = sqlite3.connect("/home/xiaoyu/IdeaProjects/argus/main.db")
cur = conn.cursor()

In [84]:
# https://www.chinamoney.com.cn/ags/ms/cm-u-bond-md/BondDetailInfo
# bondDefinedCode: dhcjeu7vld
# p = {"bondDefinedCode":"dhcjeu7vld"}


def saveDetailToDb(d):
    print("inserting",d['bondFullName'])
    fs = "'"+"','".join(d.keys())+"'"
    vs = "'"+"','".join(d.values())+"'"
    _sql = f"INSERT INTO bondDetail ({fs},'ts') VALUES ({vs},'{str(datetime.now())}')"
    print(_sql)
    cur.execute(_sql)
    conn.commit()
    
def fetchBondDetail(did,st):
    p = {"bondDefinedCode":did}
    r = requests.post(u,p)
    time.sleep(st)
    _bondDetail = json.loads(r.text)['data']
    return _bondDetail

def extractBondDetail(did):
    #r =fetchBondDetail(did)
    fs = ['bondFullName','bondCode','issueDate','mrtyDate','bondPeriod','parValue'
          ,'issuePrice','issueAmnt','frstValueDate','startenty']
    return {k:v for k,v in did['bondBaseInfo'].items() if k in set(fs)}

def listOSBondID():
    _r = cur.execute("select bondDefinedCode from issuance where bondCode not in (select bondCode from bondDetail)")
    r = [ _[0] for _ in _r.fetchall() ]
    return r



In [69]:
extractBondDetail(r)

{'bondFullName': '睿程2022年第二期个人汽车抵押贷款次级资产支持证券',
 'bondCode': '2289202',
 'issueDate': '2022-08-23',
 'mrtyDate': '2026-01-26',
 'bondPeriod': '1249日',
 'parValue': '100.00',
 'issuePrice': '100.0000',
 'issueAmnt': '3.999999864',
 'frstValueDate': '2022-08-26',
 'startenty': '北京现代汽车金融有限公司'}

In [125]:
cur.execute(f"select distinct(bondFullName) from issuance as I right join bondDetail as B on B.bondCode=I.bondCode where I.bondName like '22隆和4优先B' or I.bondCode like '22隆和4优先B';").fetchone()

('隆和2022年第四期微小企业贷款优先B级资产支持证券',)

In [126]:
def getFullName(_x):
    return cur.execute(f"select distinct(bondFullName) from issuance as I right join bondDetail as B on B.bondCode=I.bondCode where I.bondName like '{_x}' or I.bondCode like '{_x}';").fetchone()

In [127]:
getFullName('22隆和4优先B')

('隆和2022年第四期微小企业贷款优先B级资产支持证券',)

In [133]:
cur.execute(f"select distinct(bondFullName) from issuance as I left join bondDetail as B on B.bondCode=I.bondCode where I.bondName like '22隆和4优先B' or I.bondCode like '22隆和4优先B';").fetchone()

('隆和2022年第四期微小企业贷款优先B级资产支持证券',)